# Bow river above Banff
 
## Running SUMMA

<br>

## Notebook setup

In [ ]:
# modules
import os
import time
import subprocess # needed to run mizuRoute.exe from the notebook
import numpy as np
import xarray as xr
import pysumma as ps
import geopandas as gpd
import matplotlib.pyplot as plt

In [ ]:
# define the base path so that we won't have to type this over and over
path_base = '/home/jovyan/data/cewa564_bow/'

<br>

## Spatial discretization
The four different spatial discretization setups are:
- The catchment is treated as a single, lumped entity. In SUMMA terms, it consists of a single Geographical Response Unit (GRU)
- The catchment is treated as a single GRU, but subdivided into three Hydrological Response Units (HRUs) that represent different elevation bands
- The cachment is divided into 51 GRUs that represent different sub-catchments, based on the river network
- The catchment is divided into 51 sub-catchments (GRUs), where the GRUs are further subdivided into HRUs that represent different elevation bands



<br>

## Model setup
Now we'll need to setup a pysumma simulation object. You won't need to run the models for this exercise (although you can with the code at the end of this notebook if you wnat to), but the psyumma objects can be useful to access the parameter and attribute values for each model setup.

In [ ]:
# SUMMA.exe location
executable = 'summa.exe'

In [ ]:
# Define location of .exe and file manager
file_manager_lumped           = path_base + 'settings/bow/lumped/fileManager.txt'
file_manager_lumped_elev      = path_base + 'settings/bow/lumped_elevationBands/fileManager.txt'
file_manager_distributed      = path_base + 'settings/bow/distributed/fileManager.txt'
file_manager_distributed_elev = path_base + 'settings/bow/distributed_elevationBands/fileManager.txt'

In [ ]:
# Open pysumma simulations for both
s_lumped           = ps.Simulation(executable, file_manager_lumped)
s_lumped_elev      = ps.Simulation(executable, file_manager_lumped_elev)
s_distributed      = ps.Simulation(executable, file_manager_distributed)
s_distributed_elev = ps.Simulation(executable, file_manager_distributed_elev)

## Code to run these models for yourself
In case you're curious about model run times for different setups or wish to do some analysis that invovles changing model parametrizations or parameters, we here provide the code needed to run each of the four SUMMA setups. We've also included the code needed to do the routing of the two distributed setups. Note that these model setups are run in series. Parallelization is the focus of a later lecture. Enjoy!

In [ ]:
# Make the output directory if it doesn't exist
if not os.path.exists(s_lumped.manager['outputPath'].value):
    os.makedirs(s_lumped.manager['outputPath'].value)
    
if not os.path.exists(s_lumped_elev.manager['outputPath'].value):
    os.makedirs(s_lumped_elev.manager['outputPath'].value)    
    
if not os.path.exists(s_distributed.manager['outputPath'].value):
    os.makedirs(s_distributed.manager['outputPath'].value) 
    
if not os.path.exists(s_distributed_elev.manager['outputPath'].value):
    os.makedirs(s_distributed_elev.manager['outputPath'].value) 

<br>

#### Case 1: lumped catchment
In the lumped case, the catchment is treated as a single (lumped) entity. In SUMMA terms, this model setup uses a single GRU (Geographic Response Unit). No routing is needed as a result.

In [ ]:
# Initialize a timer
case1_time_start = time.time()

# Run the lumped model with default settings
s_lumped.run('local', run_suffix='default')
print(s_lumped.status)

# End the time and show result
case1_time_end = time.time()
print('Model run took ' + str(round(case1_time_end - case1_time_start)) + ' sec.')

<br>

#### Case 2: lumped catchment with elevation bands
This setup introduces some spatial complexity, by defining elevation zones within the catchment. In SUMMA terms, the single lumped GRU (see case 1) is now sub-divided into three HRUs (Hydrological Response Units). SUMMA internally handles the routing from the highest HRU to the lowest. No external routing is needed.

In [ ]:
# Initialize a timer
case2_time_start = time.time()

# Run the model with the default settings
s_lumped_elev.run('local', run_suffix='default')
print(s_lumped_elev.status)

# End the time and show result
case2_time_end = time.time()
print('Model run took ' + str(round(case2_time_end - case2_time_start)) + ' sec.')

<br>

#### Case 3: distributed catchment
This setup introduces a different way of spatial discretizaiton, namely sub-dividing a larger catchment into multiple sub-catchments. This setup uses the MERIT Hydro basin delineation (Yamazaki et al., 2019) which provides 51 sub-catchments for our modelling domain. In SUMMA terms, this model setup uses 51 GRUs (technically, the setup also includes 51 HRUs that exactly overlap the GRUs - each GRU is "divided" into 1 HRU). SUMMA does not provide GRU-to-GRU routing and external routing is needed,which we will not do here (but output for the default case is provided). 

**SUMMA**

In [ ]:
# Initialize a timer
case3_time_summa_start = time.time()

# Run the model with the default settings
s_distributed.run('local', run_suffix='default')
print(s_distributed.status)

# End the time and show result
case3_time_summa_end = time.time()
print('Model run took ' + str(round(case3_time_summa_end - case3_time_summa_start)) + ' sec.')

**mizuRoute**

Normally you would need to run a second model (mizuRoute) to create streamflow from the 51 HRUs. We will not do that here.

<br>

#### Case 4: distributed catchment with elevation bands
This setup introduces even more spatial complexity, by sub-dividing each of the 51 GRUs in up to three elevation zones. As a result each GRU can now consist of up to 3 HRUs. As before (see case 2), SUMMA handles the HRU-to-HRU routing, but external routing is needed to do the GRU-to-GRU routing and gives us simulated streamflow at the outlet.

**SUMMA**

In [ ]:
# Initialize a timer
case4_time_summa_start = time.time()

# Run the model with the default settings
s_distributed_elev.run('local', run_suffix='default')
print(s_distributed_elev.status)


# End the time and show result
case4_time_summa_end = time.time()
print('Model run took ' + str(round(case4_time_summa_end - case4_time_summa_start)) + ' sec.')

**mizuRoute**

Normally you would need to run a second model (mizuRoute) to create streamflow from the 51 HRUs. We will not do that here.